In [1]:
import pandas as pd

answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [2]:
from tqdm import tqdm 
tqdm.pandas()

In [3]:
def time_(x):
    temp=x[1:].split('-H')
    return int(temp[0])*24+int(temp[1])

In [4]:
ques_info['qtime']=ques_info['qtime'].progress_apply(lambda x:time_(x))

100%|██████████| 1829900/1829900 [00:07<00:00, 242264.96it/s]


In [5]:
answer_info['day']=answer_info['atime'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
answer_info['hour']=answer_info['atime'].progress_apply(lambda x:int(x[1:].split('-H')[1]))
answer_info['atime']=answer_info['atime'].progress_apply(lambda x:time_(x))


100%|██████████| 4513735/4513735 [00:19<00:00, 233077.39it/s]


In [6]:
answer_info=answer_info[['qid','uid','atime','day','hour']]

In [7]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])

In [8]:
all_data=pd.concat([train[['qid','uid','time']],test[['qid','uid','time']]])

In [9]:
# all_data['day']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))
# all_data['hour']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))

all_data['time']=all_data['time'].progress_apply(lambda x:time_(x))

100%|██████████| 10630845/10630845 [00:45<00:00, 232113.51it/s]


In [10]:
answer_info.columns=['aqid','uid','atime','day','hour']

In [11]:
all_data=pd.merge(all_data,answer_info,on='uid',how='inner')
all_data['sub_time']=all_data['time']-all_data['atime']
all_data=all_data[all_data['sub_time']>=0]
len(all_data)

92655969

In [12]:
all_data=all_data[all_data['qid']!=all_data['aqid']]

In [13]:
all_data.head()

,qid,uid,time,aqid,atime,day,hour,sub_time
0,Q604029601,M2317670257,92703,Q3034061280,92364,3848,12,339
2,Q604029601,M2317670257,92703,Q1309562090,92539,3855,19,164
6,Q426016802,M2317670257,92653,Q3034061280,92364,3848,12,289
8,Q426016802,M2317670257,92653,Q1309562090,92539,3855,19,114
12,Q4274079416,M2317670257,92532,Q3034061280,92364,3848,12,168


In [15]:
all_data=all_data.groupby(['uid','qid'])['aqid'].agg(['count'])

In [16]:
all_data=all_data.reset_index()

In [19]:
all_data.to_csv('feature_ua_count.csv',index=False)